In [123]:
import spiceypy as spice
import numpy as np
from datetime import datetime

In [124]:
spice.furnsh("../data/ephemeris/spice/meta/metakernel.tm")

In [125]:
time_str = "2024-01-01T00:00:00"
dt = datetime.fromisoformat(time_str)
# Convert Python datetime to a format recognized by SPICE
time_str = dt.strftime('%Y %b %d %H:%M:%S.%f')
et = spice.str2et(time_str)

observer = "0"
frame = "ECLIPJ2000"
target = "599"

state, lt = spice.spkezr("599", et, "ECLIPJ2000", 'NONE', "0")


In [126]:
# # List of all SPK files from your metakernel
# spk_files = [
#     'spk/de440.bsp',
#     'spk/ceres_1900_2100.bsp',
#     'spk/codes_300ast_20100725.bsp',
#     'spk/jupiter/jup344.bsp',
#     'spk/jupiter/jup344-s2003_j24.bsp',
#     'spk/jupiter/jup346.bsp',
#     'spk/jupiter/jup365.bsp',
#     'spk/mars/mar097.bsp',
#     'spk/neptune/nep095.bsp',
#     'spk/neptune/nep097.bsp',
#     'spk/neptune/nep097xl-801.bsp',
#     'spk/neptune/nep097xl-899.bsp',
#     'spk/neptune/nep101xl.bsp',
#     'spk/neptune/nep101xl-802.bsp',
#     'spk/neptune/nep104.bsp',
#     'spk/neptune/nep105.bsp',
#     'spk/pluto/plu060.bsp',
#     'spk/saturn/sat393_daphnis.bsp',
#     'spk/saturn/sat415.bsp',
#     'spk/saturn/sat441.bsp',
#     'spk/saturn/sat454.bsp',
#     'spk/uranus/ura111.bsp',
#     'spk/uranus/ura111l.bsp',
#     'spk/uranus/ura111xl-701.bsp',
#     'spk/uranus/ura111xl-702.bsp',
#     'spk/uranus/ura111xl-703.bsp',
#     'spk/uranus/ura111xl-704.bsp',
#     'spk/uranus/ura111xl-705.bsp',
#     'spk/uranus/ura111xl-799.bsp',
#     'spk/uranus/ura115.bsp',
#     'spk/uranus/ura116.bsp',
#     'spk/uranus/ura117.bsp',
#     'spk/comets/ison.bsp',
#     'spk/comets/siding_spring_8-19-14.bsp',
#     'spk/comets/siding_spring_s46.bsp'
# ]
#
# # Base path from your metakernel
# base_path = r'data\ephemeris\spice\kernels'
#
# # Start with set containing SSB (ID 0)
# all_ids = {0}  # Initialize with SSB
#
# # Get IDs from each SPK file and their coverage
# for kernel in spk_files:
#     try:
#         full_path = f"{base_path}/{kernel}"
#         ids = spice.spkobj(full_path)
#         for i in range(spice.card(ids)):
#             obj_id = ids[i]
#             all_ids.add(obj_id)
#
#             # Get coverage for this object
#             coverage = spice.spkcov(full_path, obj_id)
#
#             # Get number of intervals
#             niv = spice.wncard(coverage)
#
#             try:
#                 name = spice.bodc2n(obj_id)
#             except:
#                 name = "No name found"
#
#             print(f"\n{'='*50}")
#             print(f"Coverage for object {obj_id} - {name}")
#
#             # Display each coverage interval
#             for j in range(niv):
#                 # Get interval start and end times
#                 start, end = spice.wnfetd(coverage, j)
#
#                 # Convert to calendar format
#                 start_cal = spice.timout(start, "YYYY MON DD HR:MN:SC.### (TDB) ::TDB")
#                 end_cal = spice.timout(end, "YYYY MON DD HR:MN:SC.### (TDB) ::TDB")
#
#                 print(f"\nInterval: {j}")
#                 print(f"Start:    {start_cal}")
#                 print(f"Stop:     {end_cal}")
#
#     except Exception as e:
#         print(f"Skipping {kernel}: {str(e)}")

In [128]:
def calc_ephemeris( target, et, frame, observer ):
	'''
	Convenience wrapper for spkezr and spkgeo
	'''

	if type( target ) == str:
		return np.array( spice.spkezr( target, et, frame, 'NONE', observer )[ 0 ] )

	else:
		n_states = len( et )
		states   = np.zeros( ( n_states, 6 ) )
		for n in range( n_states ):
			states[ n ] = spice.spkgeo( target, et[ n ], frame, observer )[ 0 ]
		return states

val = calc_ephemeris( target, et, frame, observer )
print( val )

[ 5.21378213e+08  5.31392793e+08 -1.38692598e+07 -9.47276111e+00
  9.76970848e+00  1.71348472e-01]


In [ ]:
def state2coes( state, args = {} ):
	_args = {
		'et'        : 0,
		'mu'        : pd.earth[ 'mu' ],
		'deg'       : True,
		'print_coes': False
	}
	for key in args.keys():
		_args[ key ] = args[ key ]

	rp,e,i,raan,aop,ma,t0,mu,ta,a,T = spice.oscltx(
		state, _args[ 'et' ], _args[ 'mu' ] )

	if _args[ 'deg' ]:
		i    *= nt.r2d
		ta   *= nt.r2d
		aop  *= nt.r2d
		raan *= nt.r2d

	if _args[ 'print_coes' ]:
		print( 'a'   , a    )
		print( 'e'   , e    )
		print( 'i'   , i    )
		print( 'RAAN', raan )
		print( 'AOP' , aop  )
		print( 'TA'  , ta   )
		print()

	return [ a, e, i, ta, aop, raan ]